In [88]:
import pandas as pd
from pathlib import Path
import os
project_dir = Path(os.path.abspath('')).resolve().parents[0]

In [89]:
ratings = pd.read_csv(os.path.join(project_dir, "data/interim/ratings.csv"), index_col=0)
ratings.head()

,index,scene_id,duration,rating,sessionID
0,0,01_MS_C_344,3679.0,3.0,6bbf3fed-46ab-4221-930b-4ac3fa0acff8
1,1,01_CP_C_1226,2835.0,3.0,6bbf3fed-46ab-4221-930b-4ac3fa0acff8
2,2,01_SE_C_54,6589.0,1.0,6bbf3fed-46ab-4221-930b-4ac3fa0acff8
3,3,01_SE_C_26,2811.0,2.0,6bbf3fed-46ab-4221-930b-4ac3fa0acff8
4,4,01_MS_C_290,6401.0,1.0,6bbf3fed-46ab-4221-930b-4ac3fa0acff8


In [90]:
df = pd.DataFrame([])
means = ratings.groupby("scene_id")["rating"].mean()
stds = ratings.groupby("scene_id")["rating"].std()
counts = ratings["scene_id"].value_counts()
df["Means"] = means
df["Stds"] = stds
df["Counts"] = counts
df

,Means,Stds,Counts
scene_id,,,
01_CP_A_136,2.511111,0.548644,45
01_CP_A_137,2.289855,0.749538,69
01_CP_A_138,2.391304,0.714244,46
01_CP_A_139,2.500000,0.687730,38
01_CP_A_140,2.391304,0.802171,46
...,...,...,...
01_SE_C_86,0.544393,0.704879,428
01_SE_C_87,0.848341,0.761755,422
01_SE_C_88,0.960298,0.827639,403


In [91]:
df.to_csv(os.path.join(project_dir, "data", "interim", "means2.csv"))

In [92]:
merged_data_raw = {}
raw_scene_data = {}
drop = {}
scenes_cp = pd.read_csv(os.path.join(project_dir, "data/raw/scenes_cp.csv"),
                                header=0).rename(columns={"SceneID": "scene_id"})
raw_scene_data["CP"] = scenes_cp
drop["CP"] = ["weight", "Experiment", "Raus", "scene_id", "SR_lD"]
scenes_se = pd.read_csv(os.path.join(project_dir, "data/raw/scenes_se.csv"),
                                header=0).rename(columns={"SceneID": "scene_id"})
raw_scene_data["SE"] = scenes_se
drop["SE"] = ["weight", "Experiment", "raus", "scene_id", "NVS_lD"]
scenes_ms = pd.read_csv(os.path.join(project_dir, "data/raw/scenes_ms.csv"),
                                header=0).rename(columns={"SceneID": "scene_id"})
raw_scene_data["MS"] = scenes_ms
drop["MS"] = ["weight", "Experiment", "Raus", "scene_id", "HVS_lD"]

for experiment in raw_scene_data.keys():
    merged_df = pd.merge(ratings, raw_scene_data[experiment], how="inner", on="scene_id")
    merged_df = merged_df.drop(drop[experiment], axis=1)
    merged_data_raw[experiment] = merged_df

In [93]:
for dataset in merged_data_raw.keys():
    print(merged_data_raw[dataset].shape)

(165083, 16)
(48559, 11)
(94070, 18)


In [94]:
print(ratings.shape)

(307712, 5)


In [95]:
df = pd.DataFrame([], columns=["Experiment", "Merkmal", "Ausprägung", "Mean", "Std", "Count"])
for experiment in merged_data_raw.keys():
    current_data = merged_data_raw[experiment].copy()
    for col in raw_scene_data[experiment].drop(drop[experiment], axis=1).columns:
        unique = raw_scene_data[experiment][col].unique()
        for u in unique:
            unique_data = current_data[current_data[col] == u]["rating"]
            mean = unique_data.mean()
            std = unique_data.std()
            count = unique_data.shape[0]
            df = df.append(pd.Series([experiment, col, u, mean, std, count], index=["Experiment", "Merkmal", "Ausprägung", "Mean", "Std", "Count"]), ignore_index=True)

df

,Experiment,Merkmal,Ausprägung,Mean,Std,Count
0,CP,Kamera,C,2.245471,0.848935,86825
1,CP,Kamera,P,1.976127,0.894055,72676
2,CP,Kamera,A,2.444106,0.687782,5582
3,CP,Basisszenario,1,1.835125,0.908730,7442
4,CP,Basisszenario,2,2.045658,0.868718,53507
...,...,...,...,...,...,...
106,MS,Tr_re-Markierung,durchgezogen,1.859341,0.886722,17290
107,MS,Tr_re-Markierung,Doppellinie,1.841199,0.894600,13476
108,MS,Tr_re-Markierung,Sperrfläche,1.853739,0.887839,13387
109,MS,Parken,-,2.160772,0.904424,45151


In [96]:
df.to_csv(os.path.join(project_dir, "data", "interim", "means1.csv"))